# Rocket motor model
Это симуляция твердотопливного ракетного двигателя.
Она сделана по образу программы [Rocki-motors](http://kia-soft.narod.ru/soft/rpro/rms/rms.rar).
Дополнительно можно рассчитать полёт ракеты.

Чтобы запустить программу, задайте настройки во 1-й ячейке и запустите её кнопкой в верхнем левом углу. Потом запустите ячейку с кодом программы (последнюю). Снизу будет вывод программы.
Чтобы сменить настройки, нужно снова запустить ячейку настроек.

Если вы просматриваете этот файл на GitHub, перейдите сначала [сюда](https://colab.research.google.com/github/busyBeaver1/Rocket_motor_model/blob/main/Rocket_motor_model.ipynb), чтобы запустить програму.

!Эта программа НЕ расчитывает переходные состояния двигателя (выход на режим и сброс давления)! Однако переходные состояния обычно слабо влияют на общую картину и для большинства любителей в большинстве случаев не важны.

In [ ]:
# settings
# настройки

# параметры двигателя ==================
# двигатель состоит из цилиндрических шашек с каналом в середине. В конце, за шашками также может быть замедлитель (трассер). В трассере канала нет. Между шашками должен быть проложен мгновенный воспламеняющий состав.
# про вторичный (мгновенный) воспламеняющий состав см. здесь: https://youtu.be/mQn43RASYNM
# если этого состава нет, установите значение multi_cell False. Тогда все шашки двигателя будут работать как будто это одна большая.
multi_cell = True
d = 28. # диаметр шашки, мм
hole = 7. # диаметр канала, мм
l = 45. # длина шашки, мм
h = 2 # количество шашек
d_t = 28. # диаметр трассера (замедлителя), мм
l_t = 0. # длина трассера, мм. Если трассер не нужен, поставьте 0.
nozzle_throat = 6. # диаметр критики сопла (диаметр самого узкого места), мм
nozzle_out = 14. # диаметр выхода из сопла, мм
d_throat = 0.2 # разгар критики сопла (увеличение диаметра), мм
d_out = 0. # разгар выхода сопла, мм. Должен быть таким чтобы выход сопла НЕ стал меньше критики.

# параметры топлива ====================
# по умолчанию заданы параметры для карамельного топлива 65% KNO₃ 35% C₁₂H₂₂O₁₁ (сахароза)
a = 8.2635  # параметр, задающий скорость горения
n = 0.319 # параметр, задающий скорость горения
# скорость горения определяется как u = a * p^n, где u - скорость (мм/с), p - давление (МПа)
# значения a и n для разных топлив см. здесь: http://kia-soft.narod.ru/interests/rockets/theory/burnlow/burnlow.htm
# оттуда взята и эта формула скорости горения 
density = 1.88 # плотность топлива, г/см^3
mol = 42. # молярная масса. Масса в граммах газа И ДЫМА на 1 моль ГАЗА.
t = 1450. + 273.15 # температура горения топлива. Температура нужна в °K, поэтому к температуре в °C прибавляется 273,15

I = 5.5 # среднее количество степеней свободы молекулы газа из выхлопа движка
# для отдельных атомов I = 3, для 2-атомных молекул - 5, для 3-атомных и с большим количеством атомов - 6.
# только если связи в молекуле жёсткие (как определить - не знаю). Усреднять нужно с учётом количества молекул.
smoke = 0.439 # массовая доля дыма в выхлопе
smoke_c = 1489. # теплоёмкость дыма, Дж/(кг*K)
# I, smoke и smoke_c нужны для расчита коэффициента адиабаты k. Если k известен, просто укажите его. Тогда I, smoke и smoke_c не используются.
k = ... # коэффициент адиабаты  выхлопа

# параметры состава трассера ===========
# по умолчанию заданы параметры для карамельного топлива 65% KNO₃ 35% C₁₂H₂₂O₁₁
a_t = 8.2635
n_t = 0.319
density_t = 1.88
# отличия продуктов горения трассера и топлива не учитываются, т.к. доля продуктов сгорания трассера во всём выхлопе обычно мала
# после сгорания топлива скорость горения трассера расчитывается для атмосферного давления, что может быть некорректно, если трассер горит быстро
# поэтому используйте трассер из медленного состава или используйте другую программу для более правильного расчёта горения трассера

# поправки =============================
speed_k = 1. # отношение реальной скорости горения к теоретической
nozzle_friction = 1. # отношение реальной скорости выхода газов из сопла к теоретической. Сильно влияет на давление.
thrust_loss = 1. # отношение реальной тяги к теоретической

# другие параметры =====================
dots = 40. # точность симуляции, точек/мм
timestep = 1. / 400. # точность симуляции, длина временного шага, с
dynamic_nozzle_out = True # может ли поток газов "оторваться" от стенки сопла
out_pressure = 1.1 # давление, при котором поток отрывается от сопла, атм
# если dynamic_nozzle_out = True, значит, при слишком большом выходном диаметре сопла выхлопные газы оторвутся от сопла
# и сформируют струю раньше конца сопла, в момент когда их давление станет равным out_pressure
# тогда расчёт ведётся так, как будто выходной диаметр сопла равен тому диаметру в котором давление равно out_pressure
# если dynamic_nozzle_out = False, значит, газы продолжат идти по соплу и расширяться даже если их давление меньше атмосферного
# тогда тяга может стать даже отрицательной (из-за того что такая модель не подходит для некоторых случаев)
# как оно есть на самом деле - не знаю точно. Предположительно, если угол расширения сопла мал, а превышение оптимального выходного диаметра незначительно, dynamic_nozzle_out ближе к False
# если же выходной диаметр сопла меньше оптимального (т.е. давление у выхода сопла больше атмосферного), значение dynamic_nozzle_out ни на что не влияет.

# параметры ракеты =====================
# здесь можно рассчитать высоту полёта ракеты, указав трение и массу
friction = 0.1 # сила трения ракеты о воздух, отнощение силы трения в ньютонах к скорости в метрах в секунду, Н*с/м. Сейчас указано число просто из головы, возможно оно неправдоподобно.
mass = 1000. # масса ракеты, г

In [ ]:
# другой вариант параметров

# параметры топлива ====================
# параметры для карамельного топлива 65% KNO₃ 35% C₁₂H₂₂O₁₁ с добавкой water воды
water = .04 # доля воды в топливе
a = 8.2635  # параметр, задающий скорость горения
n = 0.319 # параметр, задающий скорость горения
# вода не учитывается в законе горения т.к. я не знаю, как её учесть
density = 1.88 * (1. - water) + 1. * water # плотность топлива, г/см^3
mol = 42. * (1. - water) + 18. * water # молярная масса. Масса в граммах газа И ДЫМА на 1 моль ГАЗА.
k = 1.14 * (1. - water) + 1.33 * water # коэффициент адиабаты  выхлопа
t = (.14 * (1450. + 273.15) * (1. - water) + .33 * (20. + 273.) * water) / (.14 * (1. - water) + .33 * water) # температура горения топлива

In [ ]:
# другой вариант параметров

# параметры топлива ====================
# параметры для карамельного топлива 65% KNO₃ 35% C₆H₁₄O₆ (сорбит)
# эти параметры полностью взяты из Rocki-motors
a = 5.132  # параметр, задающий скорость горения
n = 0.322 # параметр, задающий скорость горения
density = 1.84 # плотность топлива, г/см^3
mol = 39.9 # молярная масса. Масса в граммах газа И ДЫМА на 1 моль ГАЗА.
t = 1600 # температура горения топлива в °K. Также в Roki-motors указана "практическая температура сгорания" в 1520 °K. Если нужно, замените.
k = 1.136 # коэффициент адиабаты  выхлопа

In [ ]:
# simulation
# симуляция

import numpy, numba, math
from matplotlib import pyplot

if not type(k) in (int, float):
    gas_cv = I * .5 * 8.314 / (mol * .001 * (1. - smoke))
    gas_y = (I + 2) / I
    gas_cp = gas_y * gas_cv
    k = (gas_cp + smoke_c) / (gas_cv + smoke_c)
    print('коэффициент адиабаты:', k)

def get_grain_shape(d, hole, l):
    f = numpy.ndarray((int((d - hole) * .5 * dots), int(l * dots)), dtype='uint8')
    f[...] = 1
    if multi_cell:
        f[:, 0] = 2
        f[:, f.shape[1] - 1] = 2
    f[0, :] = 2
    return f

@numba.jit
def step(f, f1):
    n = 0.
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            if f1[i, j] == 2:
                for x, y in [(i, j-1), (i+1, j-1), (i+1, j), (i+1, j+1), (i, j+1), (i-1, j+1), (i-1, j), (i-1, j-1)]:
                    if 0 <= x < f.shape[0] and 0 <= y < f.shape[1]:
                        if f[x, y] == 1:
                            f[x, y] = 2
                n += (i * 2. + hole * dots)
                f[i, j] = 0
    return n * math.pi * .001 * dots ** (-3) * density * h

grain = get_grain_shape(d, hole, l)
x = step(grain, numpy.copy(grain))
i = 0
Y = []
while x > 0.:
    Y.append(x)
    i += 1
    x = step(grain, numpy.copy(grain))

propellant = sum(Y)
print('масса топлива:', propellant, 'г')

@numba.jit
def T2(t1, p1, p2, k): # температура на выходе
    return t1 * p2 ** (1. - 1. / k) * p1 ** (1. / k - 1.)

@numba.jit
def U2(t1, t2, mol, k, R=8.314): # скорость потока на выходе
    u22 = R / mol * 2 * k / (k - 1) * (t1 - t2)
    return math.sqrt(u22)

@numba.jit
def S1_s2(p1, p2, k): # критика / выход сопла
    a = ((k + 1.) * .5) ** (2./(k-1.)) * ((k + 1.) / (k - 1.)) / p1 ** (2./k)
    b = p2 ** (2./k) - p2 ** (1./k+1.) / p1 ** (1.-1./k)
    return (a * b) ** .5

@numba.jit
def S1_s2_d(p1, p2, k): # производная S1_s2_2 по p2
    a = ((k + 1.) * .5) ** (2./(k-1.)) * ((k + 1.) / (k - 1.)) / p1 ** (2./k)
    b = 2. / k * p2 ** (2./k-1.) - (1. + 1. / k) * p2 ** (1./k) / p1 ** (1.-1./k)
    return a * b / S1_s2(p1, p2, k) * .5

@numba.jit
def P2(p1, s1, s2, k): # давление на выходе
    p2 = 1e-40
    x = s1 / s2
    for _ in range(10):
        dx = S1_s2_d(p1, p2, k)
        p2 += (x - S1_s2(p1, p2, k)) / dx
    return p2

@numba.jit
def M(mol, p2, t2, s2, u2, R=8.314): # масса выходящих газов за секунду
    return (mol * p2 / t2 / R) * s2 * u2

@numba.jit
def P_pe_u_s2_true_t2(m, mol, s1, s2, t1, k, dynamic_nozzle_out, R=8.314): # давление и скорость выходящих газов
    p1 = 1.; s2_true = s2
    for _ in range(2):
        p2 = P2(p1, s1, s2_true, k)
        t2 = T2(t1, p1, p2, k)
        u2 = U2(t1, t2, mol, k, R=R) * nozzle_friction
        m2 = M(mol, p2, t2, s2_true, u2, R=R)
        p1 *= m / m2
        if dynamic_nozzle_out: s2_true = min(s2, s1 / S1_s2(p1, 101325. * out_pressure, k))
        else: s2_true = s2
    return p1, p2, u2, s2_true, t2

w = True
s1 = nozzle_throat ** 2 * 0.25 * math.pi * 1e-6
s2 = nozzle_out    ** 2 * 0.25 * math.pi * 1e-6
ds1 = (nozzle_throat + d_throat) ** 2 * 0.25 * math.pi * 1e-6 - s1
ds2 = (nozzle_out    + d_out   ) ** 2 * 0.25 * math.pi * 1e-6 - s2
X = [0.]; Yp = [0.]; Yt = [0.]; Yt2 = []; Ym = [0.]; Ykn = [0.]; Ykn_t = [0.]
i1 = 0.; time = 0.
l_t_2 = l_t; time_t = ...
while i1 < i:
    r = 1.; r_t = 1.; p = 1.
    for _ in range(20):
        m = 0.
        if i1 < len(Y): m += Y[int(i1)] * dots * r * speed_k
        if l_t_2 > 0.: m += d_t ** 2 * .25e-3 * math.pi * r * density_t
        p, pe, u, s2_true, t2 = P_pe_u_s2_true_t2(m * .001, mol * .001, s1, s2, t, k, dynamic_nozzle_out)
        r = a * (p * 1e-6) ** n; r_t = a_t * (p * 1e-6) ** n_t
    X.append(time)
    Yp.append(p * 9.87e-6 - 1.)
    Yt.append((u * m * .001 + s2_true * (pe - 101325.)) / 9.8 * thrust_loss)
    Yt2.append(t2)
    Ym.append(Ym[-1] - m * timestep)
    Ykn.append(Y[int(i1)] / density * dots * 1e-3 / s1)
    if l_t_2 > 0.: Ykn_t.append((Y[int(i1)] / density * dots * 1e-3 + d_t ** 2 * .25e-6 * math.pi) / s1)
    else: Ykn_t.append(Ykn[-1])
    i1 += r * dots * timestep
    if l_t_2 > 0.: l_t_2 -= r_t * timestep
    elif time_t is ...: time_t = time
    time += timestep
    s1 += ds1 * m / propellant * timestep; s2 += ds2 * m / propellant * timestep
    if not type(Yt[-1]) is float:
        if w: print('\033[31m!!!что-то не так. Проверьте параметры!!!\033[0m')
        w = False
    elif Yt[-1] < 0. and w:
        print('\033[31m!!!что-то не так. Тяга отрицательная. Проверьте параметры!!!\033[0m')
        w = False
X.append(time); Yp.append(0.); Yt.append(0.); Ym.append(Ym[-1]); Ykn.append(0.); Ykn_t.append(0.)
Ym = [m - Ym[-1] for m in Ym]

Yh = []; Xh = []
rocket_speed = 0.; height = 0.; max_speed = 0; max_g = 0.
i = 0
while rocket_speed >= 0. or i < len(Yt):
    Yh.append(height)
    Xh.append(i * timestep)
    height += rocket_speed * timestep
    ax = 0.
    if i < len(Yt):
        mass_1 = mass + Ym[i]
        ax += Yt[i] * 9.8e+3 / mass_1
    ax -= rocket_speed * friction / mass_1 * 1000. + 9.8
    rocket_speed += ax * timestep
    max_speed = max(rocket_speed, max_speed)
    max_g = max(ax / 9.8 + 1., max_g)
    i += 1

if time_t is ...: time_t = X[-1] + l_t_2 / (a_t * 101325e-6 ** n_t)
print('время работы движка:'                                          , X[-1]                                                     , 'с')
print('время горения трассера:'                                       , time_t                                                    , 'с')
print('максимальная тяга:'                                            , max(Yt)                                                   , 'кгс')
print('средняя тяга:'                                                 , sum(Yt) / len(Yt)                                         , 'кгс')
print('импульс:'                                                      , sum(Yt) * 9.8 * timestep                                  , 'кг*м/с')
print('удельный импульс:'                                             , sum(Yt) * 9.8 * timestep / propellant * 1000.             , 'м*с')
print('!давление относительно атмосферного, а не абсолютное!')
print('максимальное давление:'                                        , max(Yp)                                                   , 'атм')
print('среднее давление:'                                             , sum(Yp) / len(Yp)                                         , 'атм')
print('оптимальный диаметр выхода сопла (для среднего давления):'     , nozzle_throat / S1_s2(sum(Yp) / len(Yp) + 1., 1., k) ** .5, 'мм')
print('оптимальный диаметр выхода сопла (для максимального давления):', nozzle_throat / S1_s2(max(Yp)           + 1., 1., k) ** .5, 'мм')
print('Kn - отношение площади горения топлива к площади критики сопла')
print('максимальный Kn' + ['', ' (без трассера)'][l_t > 0.] + ':'     , max(Ykn))
print('средний Kn' + ['', ' (без трассера)'][l_t > 0.] + ':'          , sum(Ykn) / len(Ykn))
if l_t > 0.:
    print('максимальный Kn (с трассером):'                            , max(Ykn_t))
    print('средний Kn (с трассером):'                                 , sum(Ykn_t) / len(Ykn_t))
print('высота полёта:'                                                , height                                                    , 'м')
print('максимальная скорость:'                                        , max_speed                                                 , 'м/с')
print('максимальная перегрузка:'                                      , max_g                                                     , 'g')

def show(x, y, title, xlabel, ylabel):
    fig, ax = pyplot.subplots()
    ax.axhline(y=0, color='k'); ax.axvline(x=0, color='k')
    fig.suptitle(title); pyplot.xlabel(xlabel); pyplot.ylabel(ylabel)
    pyplot.plot(x, y); pyplot.show()

show(X      , Yp                         , 'Относительное давление'    , 'время, с', 'давление, атм')
show(X[1:-1], [t2 - 273.14 for t2 in Yt2], 'Температура у выхода сопла', 'время, с', 'температура, °C')
show(X      , Ykn            , 'Kn' + ['', ' (без трассера)'][l_t > 0.], 'время, с', 'Kn')
if l_t > 0.: show(X, Ykn_t               , 'Kn (с трассером)'          , 'время, с', 'Kn')
show(X      , Yt                         , 'Тяга'                      , 'время, с', 'тяга, кгс')
show(X      , Ym                         , 'Cгорание топлива'          , 'время, с', 'масса оставшегося топлива, г')
show(Xh     , Yh                         , 'Полёт'                     , 'время, с', 'высота, м')

## Ссылки
[Репозиторий с программой на GitHub](https://github.com/busyBeaver1/Rocket_motor_model)

[Закон горения](http://kia-soft.narod.ru/interests/rockets/theory/burnlow/burnlow.htm)

[Сайт автора статьи о законе горения и Rocki-motors](http://kia-soft.narod.ru/)

[Использованная здесь модель сопла - модель Nakka](https://www.nakka-rocketry.net/th_nozz.html)

[Мой канал на YouTube](https://www.youtube.com/GreatBusyBeaver/)

[Видео (не моё) про вторичный воспламеняющий состав](https://youtu.be/mQn43RASYNM)

По всем вопросам и багам пишите на почту toegorbezrukov@yandex.ru.